In [2]:
# --- Hugging Face Login & Installations ---
from google.colab import userdata
from huggingface_hub import notebook_login

hf_token = userdata.get('HF_TOKEN')
if not hf_token:
    raise ValueError("HF_TOKEN not found in Colab Secrets. Please add it.")
# notebook_login(new_session=hf_token) # Unsloth handles token auth automatically

In [3]:
# # Install Unsloth for Google Colab
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# # Standard installations
# !pip install -U transformers
# !pip install -U datasets
# !pip install -U accelerate # Required for Unsloth

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.8.1 requires msgspec, which is not installed.
unsloth-zoo 2025.8.1 requires tyro, which is not installed.
trl 0.21.0 requires transformers>=4.55.0, but you have transformers 4.54.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
from pathlib import Path
import json
import datetime
import torch
import random
import numpy as np

CONFIG = {
    # Core experiment parameters
    "experiment_type": "equation_extraction",
    # UPDATED to use Unsloth's 4-bit Gemma 3 1B model
    "model_name": "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "max_seq_length": 2048, # Unsloth's FastModel requires this at load time

    # Prompting configuration
    "include_examples": True,
    "few_shot_examples": [
        ('computational_error', 4966),
        ('conceptual_error', 1091),
    ],

    # Training parameters
    "learning_rate": 2e-4,
    "num_epochs": 1, # Set to 1 as requested
    "batch_size": 2, # Halved from 4, since Unsloth uses more VRAM initially
    "gradient_accumulation_steps": 8,

    # LoRa params (Unsloth defaults are often good)
    "lora_rank": 16,
    "lora_alpha": 16, # Often set to rank
    "lora_dropout": 0.05,

    # Paths
    "base_dataset_path": "/content/equation_extraction_dataset_cleaned.csv",
    "output_base_dir": "/content/experiments",
}

# --- Generate Unique Experiment ID ---
model_id_short = "gemma3-1b-unsloth"
experiment_id = f"{CONFIG['experiment_type']}_{model_id_short}_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"
CONFIG["experiment_id"] = experiment_id
print(f"Experiment ID: {experiment_id}")

# --- Setup Output Directories ---
output_dir = Path(CONFIG["output_base_dir"]) / CONFIG["experiment_id"]
(output_dir / "baseline_results").mkdir(parents=True, exist_ok=True)
(output_dir / "final_results").mkdir(parents=True, exist_ok=True)
CONFIG["output_dir"] = str(output_dir)
CONFIG["final_adapter_dir"] = str(output_dir / "final_adapter")
CONFIG["merged_model_dir"] = str(output_dir / "final_merged_model")

with open(output_dir / "config.json", 'w') as f: json.dump(CONFIG, f, indent=2)
print(f"Output directory created: {output_dir}")

# --- Set Random Seeds for Reproducibility ---
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seeds(42)

print("\n✅ Setup complete.")

Experiment ID: equation_extraction_gemma3-1b-unsloth_20250807_1859
Output directory created: /content/experiments/equation_extraction_gemma3-1b-unsloth_20250807_1859

✅ Setup complete.


In [6]:
### Cell 3: System Prompt for Equation Extraction

SYSTEM_PROMPT = \
"""[ROLE]
You are an expert at parsing mathematical solutions.

[TASK]
You are given a mathematical solution. Your task is to extract the calculation performed on each line and represent it as a simple equation.

For each line of the solution, create a key-value pair.
- The key should be the line identifier (e.g., "L1", "L2", "FA" for the final answer line).
- The value should be the extracted equation string (e.g., "10+5=15").
- If a line contains no calculation, the value must be an empty string.

[RESPONSE FORMAT]
Your response must be ONLY a single, valid JSON object, with no other text before or after it. The JSON object must map line identifiers to their corresponding equation strings.
"""

In [47]:
### Cell 4: Core utilities

import pandas as pd
from unsloth import FastModel
from unsloth.chat_templates import get_chat_template
import torch

# 4.1 Loading

def load_base_dataset():
    """Loads the base dataset from the specified CSV file."""
    data = pd.read_csv(CONFIG['base_dataset_path'])
    print(f"Loaded dataset with {len(data)} samples from {CONFIG['base_dataset_path']}")
    return data

def load_unsloth_model_and_tokenizer():
    """
    Loads the 4-bit quantized model and tokenizer using Unsloth's FastModel.
    """
    model_name = CONFIG["model_name"]
    print(f"Loading model and tokenizer: {model_name}")

    model, tokenizer = FastModel.from_pretrained(
        model_name = model_name,
        max_seq_length = CONFIG["max_seq_length"],
        dtype = None, # None for auto detection. Can be torch.bfloat16
        load_in_4bit = True,
    )

    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "gemma-3",
    )

    # Set padding token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = "left" # Use left-padding for generation

    return model, tokenizer

def prepare_model_for_lora_training(model):
    """Applies LoRa adapters to the model for fine-tuning."""
    print("Applying LoRa adapters to the model...")
    model = FastModel.get_peft_model(
        model,
        r = CONFIG["lora_rank"],
        lora_alpha = CONFIG["lora_alpha"],
        lora_dropout = CONFIG["lora_dropout"],
        bias = "none",
        use_gradient_checkpointing = True,
        random_state = 42,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    )
    return model

# 4.2 Formatting

def format_user_message(sample: dict) -> str:
    """
    Formats a user message to contain the literal JSON string of the answer mapping.
    - Uses 'wrong_answer_mapping' for computational or conceptual errors.
    - Uses 'correct_answer_mapping' for correct answers.
    """
    error_type = sample.get('error_type', 'correct')
    mapping_str = ""

    if error_type in ['computational_error', 'conceptual_error']:
        mapping_str = sample.get('wrong_answer_mapping', '{}')
    else:  # This covers 'correct' and any other cases
        mapping_str = sample.get('correct_answer_mapping', '{}')

    # Ensure we have a valid string to pass
    if not isinstance(mapping_str, str):
        mapping_str = '{}'

    # The prompt now contains the raw JSON string of the line-by-line solution
    return f"### Solution:\n{mapping_str.strip()}"

def format_expected_output(sample: dict) -> str:
    """
    Selects the correct equation mapping based on the error type and formats it as a clean JSON string.
    - Uses 'wrong_eqn_mapping' for computational or conceptual errors.
    - Uses 'correct_eqn_mapping' for correct answers.
    """
    error_type = sample.get('error_type', 'correct')
    eqn_map_str = ""

    if error_type in ['computational_error', 'conceptual_error']:
        eqn_map_str = sample.get('wrong_eqn_mapping', '{}')
    else: # This covers 'correct' and any other cases
        eqn_map_str = sample.get('correct_eqn_mapping', '{}')

    if not isinstance(eqn_map_str, str):
        return "{}"

    try:
        # Load and re-dump the JSON to ensure consistent, clean formatting for the model to learn.
        # This removes extra whitespace and uses double quotes.
        parsed_json = json.loads(eqn_map_str)
        return json.dumps(parsed_json, indent=2)
    except (json.JSONDecodeError, TypeError):
        # Return an empty JSON object if the source string is invalid
        return "{}"

# 4.3 Prompt Construction (CORRECTED LOGIC)

def _build_conversation_messages(sample, is_training_prompt=True):
    """
    Builds the list of messages for the chat template, correctly placing the system prompt.
    """
    # Prepend the system prompt to the user's message content.
    # This is the correct way to provide instructions for the gemma-3 template.
    user_content = format_user_message(sample)
    instructed_user_content = f"{SYSTEM_PROMPT}\n\n{user_content}"

    # For the few-shot examples, we use a simpler user message
    few_shot_messages = []
    if CONFIG["include_examples"]:
        # NOTE: We use the train_df loaded globally for this.
        example_specs = CONFIG["few_shot_examples"]
        for error_type, index in example_specs:
            example_sample_df = train_df[(train_df['error_type'] == error_type) & (train_df['index'] == index)]
            if not example_sample_df.empty:
                example_sample = example_sample_df.iloc[0].to_dict()
                # The first user turn gets the full instructions
                if not few_shot_messages:
                     few_shot_user_content = f"{SYSTEM_PROMPT}\n\n{format_user_message(example_sample)}"
                else:
                     few_shot_user_content = format_user_message(example_sample)

                few_shot_messages.append({"role": "user", "content": few_shot_user_content})
                few_shot_messages.append({"role": "assistant", "content": format_expected_output(example_sample)})

    # The final conversation list
    messages = []
    if few_shot_messages:
        messages.extend(few_shot_messages)
        # The actual query doesn't need the system prompt if few-shot examples are present
        messages.append({"role": "user", "content": format_user_message(sample)})
    else:
        # If no few-shot examples, the main user query gets the instructions
        messages.append({"role": "user", "content": instructed_user_content})

    # If it's a training prompt, add the final assistant response
    if is_training_prompt:
        messages.append({"role": "assistant", "content": format_expected_output(sample)})

    return messages

# 4.4 Prompt creation for inference
def create_sample_prompt_for_inference(sample, tokenizer):
    """Creates a full prompt for a single sample for inference."""
    messages = _build_conversation_messages(sample, is_training_prompt=False)
    # Apply the chat template with the generation prompt
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [48]:
### Cell 5: Dataset preparation

from datasets import Dataset

def prepare_datasets(base_df):
    """
    Splits the base DataFrame into training and testing sets using the 'split' column.
    """
    train_df = base_df[base_df['split'] == 'train'].copy()
    test_df = base_df[base_df['split'] == 'test'].copy()
    print(f"Data split using 'split' column: {len(train_df)} training, {len(test_df)} testing samples.")
    return train_df, test_df

def create_training_dataset(df, tokenizer):
    """
    Creates the tokenized training dataset object for the SFTTrainer.
    """
    def create_text_for_sample(sample):
        """Prepares the full conversation text for a single training sample."""
        messages = _build_conversation_messages(sample, is_training_prompt=True)
        return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

    return Dataset.from_pandas(df).map(create_text_for_sample, load_from_cache_file=False)

In [49]:
### Cell 6: Evaluation logic

import re
import math
import numpy as np
from transformers import EvalPrediction


def rigorous_compare_equation_dicts(predicted_dict: dict, expected_dict: dict) -> float:
    """
    Compares two equation dictionaries based on evaluation, operand sequence,
    and operator sequence.

    Args:
        predicted_dict: Dictionary of predicted equation strings.
        expected_dict: Dictionary of expected equation strings.

    Returns:
        A float score [0.0, 1.0] representing the mean structural accuracy.
    """

    def _safe_eval(expression: str):
        """Safely evaluates a string, returning a sentinel on error."""
        try:
            return eval(expression, {"__builtins__": None}, {})
        except Exception:
            return object()

    def _extract_components(expression: str) -> tuple[list, list]:
        """Extracts number and operator sequences from an expression string."""
        # Regex to find numbers (integers or floats)
        numbers = [float(n) for n in re.findall(r'\d+\.?\d*|\.\d+', expression)]
        # Regex to find basic arithmetic operators
        operators = re.findall(r'[+\-*/]', expression)
        return numbers, operators

    def _expressions_are_equivalent(pred_expr: str, exp_expr: str) -> bool:
        """
        Checks if two expression strings are equivalent based on:
        1. Evaluated value.
        2. Sequence of numbers.
        3. Sequence of operators.
        """
        # 1. Check if they evaluate to the same value
        pred_val = _safe_eval(pred_expr)
        exp_val = _safe_eval(exp_expr)

        if not (isinstance(pred_val, (int, float)) and isinstance(exp_val, (int, float))):
            return False
        if not math.isclose(pred_val, exp_val):
            return False

        # 2 & 3. Check for same sequence of numbers and operators
        pred_nums, pred_ops = _extract_components(pred_expr)
        exp_nums, exp_ops = _extract_components(exp_expr)

        if pred_nums != exp_nums or pred_ops != exp_ops:
            return False

        return True

    # --- Main function logic ---
    items_to_score = [(k, v) for k, v in expected_dict.items() if v]
    if not items_to_score:
        return 1.0 if not any(v for v in predicted_dict.values() if isinstance(v, str) and v) else 0.0

    scores = []
    for key, expected_eqn in items_to_score:
        line_score = 0
        predicted_eqn = predicted_dict.get(key)

        if isinstance(predicted_eqn, str) and predicted_eqn.count('=') == 1:
            pred_lhs, pred_rhs = [s.strip() for s in predicted_eqn.split('=', 1)]
            exp_lhs, exp_rhs = [s.strip() for s in expected_eqn.split('=', 1)]

            # Compare LHS and RHS using the rigorous check
            if _expressions_are_equivalent(pred_lhs, exp_lhs) and \
               _expressions_are_equivalent(pred_rhs, exp_rhs):
                line_score = 1

        scores.append(line_score)

    return sum(scores) / len(scores)

def extract_json_from_response(response: str) -> dict:
    """Extracts a JSON object from a model's text response."""
    match = re.search(r'```json\s*(\{.*?\})\s*```', response, re.DOTALL)
    json_str = match.group(1) if match else re.search(r'(\{.*?\})', response, re.DOTALL)
    if not json_str: return {}
    try:
        # For gemma, which might output python dicts
        cleaned_str = json_str.group(0).replace("'", '"') if hasattr(json_str, 'group') else json_str.replace("'", '"')
        return json.loads(cleaned_str)
    except (json.JSONDecodeError, AttributeError):
        return {}

def run_unsloth_inference(model, tokenizer, df_to_eval, batch_size=32):
    """
    Runs inference using the provided Unsloth model and tokenizer.
    """

    print(f"\n--- Running Unsloth native inference ---")

    # Prepare all prompts first, using the corrected prompt creation logic
    prompts = [
        create_sample_prompt_for_inference(row, tokenizer)
        for _, row in df_to_eval.iterrows()
    ]

    all_predictions = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Inference Batches"):
        batch_prompts = prompts[i:i + batch_size]
        # print(f"Before batch {i} input: {tokenizer.padding_side}")
        tokenizer.padding_side = "left"
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True).to("cuda")
        # print(f"Before batch {i} output: {tokenizer.padding_side}")
        tokenizer.padding_side = "left"
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id)
        # print(f"After batch {i} output: {tokenizer.padding_side}")
        tokenizer.padding_side = "left"
        decoded_outputs = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)
        all_predictions.extend(decoded_outputs)

    return all_predictions


def evaluate_predictions(test_df, predictions):
    """Parses predictions and computes metrics."""
    results_data = []
    for i, pred_text in enumerate(predictions):
        original_sample = test_df.iloc[i].to_dict()
        predicted_json = extract_json_from_response(pred_text)
        expected_json = json.loads(format_expected_output(original_sample))

        score = rigorous_compare_equation_dicts(predicted_json, expected_json)
        results_data.append({
            'problem_index': original_sample.get('index'), 'rigorous_score': score,
            'expected_json': json.dumps(expected_json), 'predicted_json': json.dumps(predicted_json),
            'full_prediction_text': pred_text.strip(),
        })
    results_df = pd.DataFrame(results_data)

    # Compute final metrics dictionary
    parse_failures = (results_df['predicted_json'] == '{}').sum()
    metrics = {
        "mean_rigorous_score": results_df['rigorous_score'].mean(),
        "total_samples": len(results_df),
        "json_parse_failures": int(parse_failures),
        "failure_rate": parse_failures / len(results_df) if len(results_df) > 0 else 0
    }
    return results_df, metrics

In [55]:
### Cell 7: Fine-tuning function

from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
from unsloth.chat_templates import train_on_responses_only

def run_fine_tuning(model, tokenizer, train_dataset):
    """Runs fine-tuning using Unsloth and SFTTrainer."""

    # Configure the trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        dataset_text_field="text",
        max_seq_length=CONFIG["max_seq_length"],
        dataset_num_proc=2,
        args=SFTConfig(
            per_device_train_batch_size=CONFIG["batch_size"],
            gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
            warmup_steps=5,
            num_train_epochs=2,
            learning_rate=CONFIG["learning_rate"],
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=42,
            output_dir=str(Path(CONFIG["output_dir"]) / "training_checkpoints"),
            report_to="none",
        ),
    )

    # Use Unsloth's helper to only train on assistant's responses
    # This is more efficient than manual masking.
    trainer = train_on_responses_only(
        trainer,
        instruction_part="<start_of_turn>user",
        response_part="<start_of_turn>model",
    )

    print(f"\n--- Starting fine-tuning for {CONFIG['num_epochs']} epoch(s) ---")
    trainer_stats = trainer.train()

    # Save the final LoRa adapter
    print(f"\n✅ Fine-tuning finished! Saving final adapter to {CONFIG['final_adapter_dir']}")
    model.save_pretrained(CONFIG["final_adapter_dir"])

    return trainer_stats

In [11]:
### Cell 8: Pipeline execution

In [50]:
# 8.1 Load dataset and few-shot examples
base_df = load_base_dataset()
train_df, test_df = prepare_datasets(base_df)
print("\n✅ Data loaded and split.")

Loaded dataset with 877 samples from /content/equation_extraction_dataset_cleaned.csv
Data split using 'split' column: 701 training, 176 testing samples.

✅ Data loaded and split.


In [38]:
# 8.2 Load model and tokenizer
model, tokenizer = load_unsloth_model_and_tokenizer()
print("\n✅ Unsloth model and tokenizer loaded.")

Loading model and tokenizer: unsloth/gemma-3-1b-it-unsloth-bnb-4bit
==((====))==  Unsloth 2025.8.1: Fast Gemma3 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

✅ Unsloth model and tokenizer loaded.


In [51]:
# 8.3 Apply Formatting (Inspect Message List)
inspection_sample = train_df.iloc[0].to_dict()

conversation_messages = _build_conversation_messages(sample=inspection_sample, is_training_prompt=True)

import json
print("Example conversation:")
for message in conversation_messages:
    print(f"{message['role']}:")
    print(message['content'])
    print()

Example conversation:
user:
[ROLE]
You are an expert at parsing mathematical solutions.

[TASK]
You are given a mathematical solution. Your task is to extract the calculation performed on each line and represent it as a simple equation.

For each line of the solution, create a key-value pair.
- The key should be the line identifier (e.g., "L1", "L2", "FA" for the final answer line).
- The value should be the extracted equation string (e.g., "10+5=15").
- If a line contains no calculation, the value must be an empty string.

[RESPONSE FORMAT]
Your response must be ONLY a single, valid JSON object, with no other text before or after it. The JSON object must map line identifiers to their corresponding equation strings.


### Solution:
{
  "L1": "2% of $90 is (2/100)*$90 = $1.8",
  "L2": "2% of $60 is (2/100)*$60 = $1.2",
  "L3": "The second transaction was reversed without the service charge so only a total of $90+$1.8+$1.2 = $39 was deducted from his account",
  "L4": "He will have a bal

In [52]:
# 8.4 Apply tokenizer and inspect

final_prompt_string = tokenizer.apply_chat_template(
    conversation_messages,
    tokenize=False,
    add_generation_prompt=False # False because it's a training example
)

print(final_prompt_string)

<bos><start_of_turn>user
[ROLE]
You are an expert at parsing mathematical solutions.

[TASK]
You are given a mathematical solution. Your task is to extract the calculation performed on each line and represent it as a simple equation.

For each line of the solution, create a key-value pair.
- The key should be the line identifier (e.g., "L1", "L2", "FA" for the final answer line).
- The value should be the extracted equation string (e.g., "10+5=15").
- If a line contains no calculation, the value must be an empty string.

[RESPONSE FORMAT]
Your response must be ONLY a single, valid JSON object, with no other text before or after it. The JSON object must map line identifiers to their corresponding equation strings.


### Solution:
{
  "L1": "2% of $90 is (2/100)*$90 = $1.8",
  "L2": "2% of $60 is (2/100)*$60 = $1.2",
  "L3": "The second transaction was reversed without the service charge so only a total of $90+$1.8+$1.2 = $39 was deducted from his account",
  "L4": "He will have a balanc

In [53]:
# ===================================================================
# PHASE 1: BASELINE EVALUATION
# ===================================================================

from tqdm import tqdm

print("\n" + "="*50)
print("PHASE 1: BASELINE EVALUATION")
print("="*50)

# Run inference on the base model
baseline_predictions = run_unsloth_inference(
    model=model,
    tokenizer=tokenizer,
    df_to_eval=test_df
)

# Evaluate and save baseline results
baseline_results_df, baseline_metrics = evaluate_predictions(test_df, baseline_predictions)
baseline_results_path = Path(CONFIG["output_dir"]) / "baseline_results" / "baseline_evaluation_results.csv"
baseline_metrics_path = Path(CONFIG["output_dir"]) / "baseline_results" / "baseline_metrics.json"
baseline_results_df.to_csv(baseline_results_path, index=False)
with open(baseline_metrics_path, 'w') as f:
    json.dump(baseline_metrics, f, indent=2)

print("\n--- Baseline Metrics ---")
print(json.dumps(baseline_metrics, indent=2))
print(f"✅ Baseline results saved.")



PHASE 1: BASELINE EVALUATION

--- Running Unsloth native inference ---


Inference Batches: 100%|██████████| 6/6 [02:13<00:00, 22.17s/it]


--- Baseline Metrics ---
{
  "mean_rigorous_score": 0.23005952380952382,
  "total_samples": 176,
  "json_parse_failures": 3,
  "failure_rate": 0.017045454545454544
}
✅ Baseline results saved.


In [54]:
baseline_results_df

,problem_index,rigorous_score,expected_json,predicted_json,full_prediction_text
0,2680,0.000000,"{""L1"": ""6/12=0.5"", ""L2"": ""2+0.5=2.5"", ""L3"": ""2...","{""L1"": ""6/12=0.5 years"", ""L2"": ""2+0.5=2.5 year...","```json\n{\n ""L1"": ""6/12=0.5 years"",\n ""L2"":..."
1,4479,0.250000,"{""L1"": ""21/14=1.5"", ""L2"": ""8+3+12+1.5=24.5"", ""...","{""L1"": ""1.5 bushels of corn."", ""L2"": ""24.5 bus...","```json\n{\n ""L1"": ""1.5 bushels of corn."",\n ..."
2,5650,0.000000,"{""L1"": ""75/60=1.25"", ""L2"": ""40*1.25=50"", ""L3"":...","{""L1"": ""1.25 pages/minute * 75 = 93.75"", ""L2"":...","```json\n{\n ""L1"": ""1.25 pages/minute * 75 = ..."
3,7201,0.000000,"{""L1"": ""200/2=100"", ""L2"": ""100*0.3=30"", ""L3"": ...","{""L1"": ""100 visitors"", ""L2"": ""210"", ""L3"": ""210...","```json\n{\n ""L1"": ""100 visitors"",\n ""L2"": ""..."
4,2796,0.250000,"{""L1"": ""0.4*80=32"", ""L2"": ""2*32=64"", ""L3"": ""10...","{""L1"": ""32"", ""L2"": ""64"", ""L3"": ""100-64=36"", ""L...","```json\n{\n ""L1"": ""32"",\n ""L2"": ""64"",\n ""L..."
...,...,...,...,...,...
171,146,0.000000,"{""L1"": ""22+3=25"", ""L2"": ""25-4=21"", ""L3"": ""22+2...","{""L1"": ""25 minutes"", ""L2"": ""21 minutes"", ""L3"":...","```json\n{\n ""L1"": ""25 minutes"",\n ""L2"": ""21..."
172,4165,0.250000,"{""L1"": ""2*2=4"", ""L2"": ""2*2*2=8"", ""L3"": ""2+4+8=...","{""L1"": ""2 x 2 = 4"", ""L2"": ""2 x 2 x 2 = 8"", ""L3...","```json\n{\n ""L1"": ""2 x 2 = 4"",\n ""L2"": ""2 x..."
173,1959,0.000000,"{""L1"": ""11-2=9"", ""L2"": ""9-3=6"", ""FA"": """"}","{""L1"": ""9 - 2 = 7"", ""L2"": ""6"", ""FA"": ""6""}","```json\n{\n ""L1"": ""9 - 2 = 7"",\n ""L2"": ""6"",..."
174,4877,0.500000,"{""L1"": ""12*2=24"", ""L2"": ""8-2=6"", ""L3"": ""6+8+6=...","{""L1"": ""12 * 2 = 24"", ""L2"": ""24 + 8 - 2 = 20"",...","```json\n{\n ""L1"": ""12 * 2 = 24"",\n ""L2"": ""2..."


In [56]:
# ===================================================================
# PHASE 2: FINE-TUNE
# ===================================================================

print("\n" + "="*50)
print("PHASE 2: FINE-TUNING")
print("="*50)

# 1. Apply LoRa adapters to the existing model object for training
model = prepare_model_for_lora_training(model)

# 2. Prepare the Hugging Face Dataset for the trainer
train_dataset = create_training_dataset(train_df, tokenizer)

# 3. Run the fine-tuning process
training_stats = run_fine_tuning(model, tokenizer, train_dataset)

print("✅ Fine-tuning complete. The model object in memory is now updated.")

# 4. Save Training Log and Configuration
log_history = [log for log in training_stats.log_history if 'loss' in log]
log_history_df = pd.DataFrame(log_history)
log_path = output_dir / "training_log.csv"
log_history_df.to_csv(log_path, index=False)
print(f"✅ Training log saved to: {log_path}")


PHASE 2: FINE-TUNING
Applying LoRa adapters to the model...


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/701 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 701 | Num Epochs = 2 | Total steps = 88
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.787500
2,0.735800
3,0.726200
4,0.712000
5,0.656000
6,0.544100
7,0.386500
8,0.319400
9,0.251700
10,0.216000



✅ Fine-tuning finished! Saving final adapter to /content/experiments/equation_extraction_gemma3-1b-unsloth_20250807_1859/final_adapter
✅ Fine-tuning complete. The model object in memory is now updated.


AttributeError: 'TrainOutput' object has no attribute 'log_history'

In [57]:
# ===================================================================
# PHASE 3: FINAL EVALUATION
# ===================================================================

import gc

print("\n" + "="*50)
print("PHASE 3: FINAL EVALUATION")
print("="*50)

# Run inference with the fine-tuned LoRa model
final_predictions = run_unsloth_inference(
    model=model,
    tokenizer=tokenizer,
    df_to_eval=test_df
)

# Evaluate and save final results
final_results_df, final_metrics = evaluate_predictions(test_df, final_predictions)
final_results_path = Path(CONFIG["output_dir"]) / "final_results" / "final_evaluation_results.csv"
final_metrics_path = Path(CONFIG["output_dir"]) / "final_results" / "final_metrics.json"
final_results_df.to_csv(final_results_path, index=False)
with open(final_metrics_path, 'w') as f:
    json.dump(final_metrics, f, indent=2)

print("\n--- Final Metrics ---")
print(json.dumps(final_metrics, indent=2))
print(f"✅ Final results saved.")

# --- Clean up ---
del model, tokenizer, final_predictions, final_results_df
gc.collect()
torch.cuda.empty_cache()


PHASE 3: FINAL EVALUATION

--- Running Unsloth native inference ---


Inference Batches: 100%|██████████| 6/6 [02:47<00:00, 27.97s/it]



--- Final Metrics ---
{
  "mean_rigorous_score": 0.9310673701298703,
  "total_samples": 176,
  "json_parse_failures": 0,
  "failure_rate": 0.0
}
✅ Final results saved.


In [58]:
# --- FINAL COMPARISON ---
print("\n" + "="*50)
print("PERFORMANCE COMPARISON")
print("="*50)

print("\n--- Baseline Metrics ---")
print(json.dumps(baseline_metrics, indent=2))

print("\n--- Final Fine-Tuned Metrics ---")
print(json.dumps(final_metrics, indent=2))
print("\n" + "="*50)

print("\n✅✅✅ Experiment Complete! ✅✅✅")


PERFORMANCE COMPARISON

--- Baseline Metrics ---
{
  "mean_rigorous_score": 0.23005952380952382,
  "total_samples": 176,
  "json_parse_failures": 3,
  "failure_rate": 0.017045454545454544
}

--- Final Fine-Tuned Metrics ---
{
  "mean_rigorous_score": 0.9310673701298703,
  "total_samples": 176,
  "json_parse_failures": 0,
  "failure_rate": 0.0
}


✅✅✅ Experiment Complete! ✅✅✅


In [59]:
# --- Final Clean up ---
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [60]:
import zipfile
from pathlib import Path
import os

print("\n" + "="*50)
print("COMPRESSING RESULTS FOR DOWNLOAD")
print("="*50)

# Define paths from the global CONFIG
output_dir = Path(CONFIG["output_dir"])
experiment_id = CONFIG["experiment_id"]
adapter_path = Path(CONFIG["final_adapter_dir"])

# Define the name and location of the output zip file
zip_path = output_dir / f"{experiment_id}_results.zip"

# List of files and directories to be included in the zip archive
files_to_zip = [
    output_dir / "baseline_results" / "baseline_evaluation_results.csv",
    output_dir / "baseline_results" / "baseline_metrics.json",
    output_dir / "final_results" / "final_evaluation_results.csv",
    output_dir / "final_results" / "final_metrics.json",
    output_dir / "training_log.csv",
    output_dir / "config.json",
]

try:
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        print(f"Creating zip archive at: {zip_path}")
        for file_path in files_to_zip:
            if file_path.exists():
                # The arcname is the path of the file relative to the experiment directory,
                # which keeps the folder structure (e.g., 'baseline_results/...') inside the zip.
                arcname = file_path.relative_to(output_dir)
                zipf.write(file_path, arcname)
                print(f"  - Adding: {arcname}")
            else:
                print(f"  - Skipping (not found): {file_path}")

    print(f"\n✅ Successfully created results zip archive at: {zip_path}")

except Exception as e:
    print(f"\n❌ An error occurred while creating the zip file: {e}")


COMPRESSING RESULTS FOR DOWNLOAD
Creating zip archive at: /content/experiments/equation_extraction_gemma3-1b-unsloth_20250807_1859/equation_extraction_gemma3-1b-unsloth_20250807_1859_results.zip
  - Adding: baseline_results/baseline_evaluation_results.csv
  - Adding: baseline_results/baseline_metrics.json
  - Adding: final_results/final_evaluation_results.csv
  - Adding: final_results/final_metrics.json
  - Skipping (not found): /content/experiments/equation_extraction_gemma3-1b-unsloth_20250807_1859/training_log.csv
  - Adding: config.json

✅ Successfully created results zip archive at: /content/experiments/equation_extraction_gemma3-1b-unsloth_20250807_1859/equation_extraction_gemma3-1b-unsloth_20250807_1859_results.zip
